In [18]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (3, 3)

from matscipy.dislocation import get_elastic_constants, plot_vitek

from ovito.io.ase import ase_to_ovito
from ovito.modifiers import CommonNeighborAnalysisModifier, IdentifyDiamondModifier
from ovito.pipeline import StaticSource, Pipeline

# Get the results of Ovito Common Neighbor Analysis 
# https://www.ovito.org/docs/current/reference/pipelines/modifiers/common_neighbor_analysis.html
# and Identify Diamond modifier
# https://www.ovito.org/docs/current/reference/pipelines/modifiers/identify_diamond.html
# for better visualisation of the dislocation core
# it will be identified as "other" structure type
def get_structure_types(structure, diamond_structure=False):
    """Get the results of Common Neighbor Analysis and 
        Identify Diamond modifiers from Ovito
    Args:
        structure (ase.atoms): input structure
    Returns:
        atom_labels (array of ints): per atom labels of the structure types
        structure_names (list of strings): names of the structure types
        colors (list of strings): colors of the structure types in hex format
    """
    ovito_structure = structure.copy()
    if "fix_mask" in ovito_structure.arrays:
        del ovito_structure.arrays["fix_mask"]
    
    if diamond_structure:
        modifier = IdentifyDiamondModifier()
    else:
        modifier = CommonNeighborAnalysisModifier() 
    
    data = ase_to_ovito(ovito_structure)
    pipeline = Pipeline(source=StaticSource(data=data))
    pipeline.modifiers.append(modifier)
    data = pipeline.compute()

    atom_labels = data.particles['Structure Type'].array

    structure_names = [structure.name for structure in modifier.structures]
    colors = [structure.color for structure in modifier.structures]
    hex_colors = ['#{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255)) for r, g, b in colors] 

    return atom_labels, structure_names, hex_colors

# interactive visualisation  inside the notebook with nglview
from nglview import show_ase, ASEStructure

def add_dislocation(view, system, name, color=[0, 1, 0], x_shift=0.0):
    '''Add dislocation line to the view as a cylinder and two cones.
    The cylinder is hollow by default so the second cylinder is needed to close it.
    In case partial distance is provided, two dislocation lines are added and are shifter accordingly.
    '''
    
    center = system.positions.mean(axis=0)
    view.shape.add_cylinder((center[0] + x_shift, center[1], -2.0), 
                            (center[0] + x_shift, center[1], system.cell[2][2] - 0.5),
                            color,
                            [0.3],
                            name)
    
    view.shape.add_cone((center[0] + x_shift, center[1], -2.0), 
                        (center[0] + x_shift, center[1], 0.5),
                        color,
                        [0.3],
                        name)
       
    view.shape.add_cone((center[0] + x_shift, center[1], system.cell[2][2] - 0.5), 
                        (center[0] + x_shift, center[1], system.cell[2][2] + 1.0),
                        color,
                        [0.55],
                        name)
    

def interactive_view(system, scale=0.5, diamond_structure=False, add_bonds=False,
                     d_name='', d_color=[0, 1, 0], partial_distance=None):

    atom_labels, structure_names, colors = get_structure_types(system, 
                                                               diamond_structure=diamond_structure)

    view = show_ase(system)
    view.hide([0])
    
    if add_bonds: # add bonds between all atoms to have bonds between structures
        component = view.add_component(ASEStructure(system), default_representation=False, name='between structures')
        component.add_ball_and_stick(cylinderOnly=True, radiusType='covalent', radiusScale=scale, aspectRatio=0.1)
    
    for structure_type in np.unique(atom_labels):
        # every structure type is a different component
        mask = atom_labels == structure_type
        component = view.add_component(ASEStructure(system[mask]), 
                                       default_representation=False, name=str(structure_names[structure_type]))
        if add_bonds:
            component.add_ball_and_stick(color=colors[structure_type], radiusType='covalent', radiusScale=scale)
        else:
            component.add_spacefill(color=colors[structure_type], radiusType='covalent', radiusScale=scale)
        
    component.add_unitcell()

    if partial_distance is None:
        add_dislocation(view, system, d_name + ' dislocation line', d_color)
    else:
        if type(d_name) == list:
            add_dislocation(view, system, d_name[0] + ' dislocation line', d_color, x_shift= -partial_distance / 2.0)
            add_dislocation(view, system, d_name[1] + ' dislocation line', d_color, x_shift= partial_distance / 2.0)
        else:
            add_dislocation(view, system, d_name + ' dislocation line', d_color, x_shift= -partial_distance / 2.0)
            add_dislocation(view, system, d_name + ' dislocation line', d_color, x_shift= partial_distance / 2.0)

    view.camera = 'orthographic'
    view.parameters = {"clipDist": 0}

    view._remote_call("setSize", target="Widget", args=["700px", "400px"])
    view.center()
    
    tooltip_js = """
                 this.stage.mouseControls.add('hoverPick', (stage, pickingProxy) => {
                        let tooltip = this.stage.tooltip;
                        if(pickingProxy && pickingProxy.atom && !pickingProxy.bond){
                            let atom = pickingProxy.atom;
                            tooltip.innerText = atom.atomname + " atom: " + atom.structure.name;
                        } else if (pickingProxy && pickingProxy.bond){
                            let bond = pickingProxy.bond;
                            tooltip.innerText = bond.atom1.atomname + "-" + bond.atom2.atomname + " bond: " + bond.structure.name;
                        } else if (pickingProxy && pickingProxy.unitcell){
                            tooltip.innerText = "Unit cell";
                        }
                    });
                 """
    view._js(tooltip_js)
    return view


# Building cylindrical configurations with dislocations

![](dislocation_cylinder.png)

The `matscipy.dislocation` provides a set of tools to create atomistic configurations containing dislocations. In this example we focus on cylindrical configurations. In order to create a dislocation we start with a cylinder of bulk material oriented along the vector of the future dislocation line $\vec l$. By convention assumed in `matscipy.dislocation` Z direction of the cell always corresponds to the dislocation line $\vec l$. Then a displacement field is applied to the cylinder and outer radial shell is fixed in order to stabilise the configuration. So we end up with periodic boundary conditions along the dislocation line (Z) and fixed boundaries along X and Y directions. 

Ase shown on the figure above, screw dislocations are characterised by a displacement with corresponding burgers vector $\vec b$ parallel to the dislocation line $\vec l$, while edge dislocations have the burgers vector $\vec b$ perpendicular to $\vec l$. Here we use an anisotrpoic solution within [Stroh formalism](https://doi.org/10.1080/14786435808565804) to generate displacement field as implemented in [atomman](https://www.ctcms.nist.gov/potentials/atomman/) package. For the visualisation of the structures we will use [Common Neighbour Analysis algoritm](https://www.ovito.org/docs/current/reference/pipelines/modifiers/common_neighbor_analysis.html) and its [extention for diamond structure](https://www.ovito.org/docs/current/reference/pipelines/modifiers/identify_diamond.html) implemented in [OVITO](https://www.ovito.org/docs/current/introduction.html) package. [OVITO](https://www.ovito.org/docs/current/introduction.html) and [atomman](https://www.ctcms.nist.gov/potentials/atomman/) are not part of the default dependencies of `matscipy` and require separate installation. Keep that in mind while running [`test_dislocation.py`](https://github.com/libAtoms/matscipy/blob/master/tests/test_dislocation.py) since the majority of the tests there require this modules and thus will be skipped if the modules are not found in your system. Please note that for this example we use very small systems (small radius). For production calculations one should do a convergence tests with the system size in order to ensure that fixed boundaries do not affect the studied properties.


## Body Centered Cubic 

Here we will use tungsten as an example BCC material. We need lattice parameter 
create dislocation configuration. For the case of _ab initio_ calculation one can provide corresponding values directly. For much faster calculations based on interatomic potentials `matscipy.dislocation` provides a convenient method `get_elastic_constants` to calculate desired properties. We will use and Embedded Atom Potential from [Marinica _et. al._ 2013 paper](http://dx.doi.org/10.1088/0953-8984/25/39/395502) (version EAM4) for the case of tungsten.

In [13]:
# the calculator to provide forces and energies from the potential
from matscipy.calculators.eam import EAM
eam_calc = EAM("../../tests/w_eam4.fs")

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=eam_calc, symbol="W")

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 14:15:21      -17.792788*       1.5616
FIRE:    1 14:15:21      -17.796007*       1.2330
FIRE:    2 14:15:21      -17.798904*       0.6354
FIRE:    3 14:15:21      -17.799853*       0.2223
FIRE:    4 14:15:21      -17.799882*       0.1976
FIRE:    5 14:15:21      -17.799928*       0.1509
FIRE:    6 14:15:21      -17.799970*       0.0870
FIRE:    7 14:15:21      -17.799991*       0.0130
FIRE:    8 14:15:21      -17.799981*       0.0629
FIRE:    9 14:15:21      -17.799981*       0.0611
FIRE:   10 14:15:21      -17.799982*       0.0576
FIRE:   11 14:15:21      -17.799984*       0.0525
FIRE:   12 14:15:21      -17.799986*       0.0458
FIRE:   13 14:15:21      -17.799988*       0.0379
FIRE:   14 14:15:21      -17.799989*       0.0289
FIRE:   15 14:15:21      -17.799991*       0.0191
FIRE:   16 14:15:21      -17.799991*       0.0076
FIRE:   17 14:15:21      -17.799992*       0.0052
FI

In [14]:
print(f"{alat:.3f} (Angstrom), {C11:.2f}, {C12:.2f}, {C44:.2f} (GPa)")

3.143 (Angstrom), 538.90, 219.93, 178.80 (GPa)


The values are in good agreement with corresponding numbers in the paper: 3.143, 523, 202, 161

### 1/2<111>{110} screw dislocation

The notation 1/2<111>{110} means a dislocation with a burgers vector 1/2<111> and glide plane {110}. For screw dislocation, the dislocation line is parallel to the burgers vector, thus Z direction of the cell will be along <111>. The glide plane {110} fixes the Y direction to be <110> and the X direction is fixed by the geometry to be along <112>. However you do not have to think about it to start since `BCCScrew111Dislocation` object contains all the necessary information. All you need to do is to pass $a_0$ and elastic constants $C_{11}$, $C_{12}$ and $C_{44}$ and the symbol of the desired BCC element to the constructor.

In [15]:
from matscipy.dislocation import BCCScrew111Dislocation

W_screw = BCCScrew111Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_screw.axes)

print("Burgers vector:")
print(W_screw.burgers)


Cell orientation:
[[ 1  1 -2]
 [-1  1  0]
 [ 1  1  1]]
Burgers vector:
[1.57169453 1.57169453 1.57169453]


We are going to use `BCCScrew111Dislocation.build_cylinder()` method to build the cells. All the necessary parameters are passed automatically and we only need to chose the radius of the cell in (X,Y) plane. The function returns reference bulk cell and a cell containing dislocation. The displacement field is applied iteratively untill a converged value is achieved, the printed output prints the difference between the steps. The radius provided is a parameter is a radius of unconstrained region around the core. The outer shell of fixed atoms is added on top of this. The default value is 10 Angstom which is about two times larger than a cutoff for the used interatomic potential.

In [16]:
W_screw_bulk, W_screw_dislo = W_screw.build_cylinder(radius=20)

print("\nCell vectors:")
print(W_screw_dislo.cell.array)

print("\nBurgers vector lenth:")
print(np.linalg.norm(W_screw.burgers))

interactive_view(W_screw_dislo, d_name="1/2<111> screw")

disloc SCF 0 |d1-d2|_inf = 0.001400723332078968
disloc SCF 1 |d1-d2|_inf = 6.704394990092455e-06
disloc SCF 2 |d1-d2|_inf = 3.2089362583809233e-08

Cell vectors:
[[61.59759392  0.          0.        ]
 [ 0.         62.235928    0.        ]
 [ 0.          0.          2.72225477]]

Burgers vector lenth:
2.722254772666657


NGLWidget()


Here we have a cell with $|\vec b| = \frac{\sqrt{3}}{2}a_0$ length along the dislocation line. With a periodic conditions along Z it corresponds to infinitely long straight dislocation line. Note that the periodic boundary conditions along the dislocation line are not applied in the visualisation. It makes it easier to see the displecement, but you might notice the atoms outside the simulation box. 

To increase the size of the cell along the line we can rely on the magic of [ASE](https://wiki.fysik.dtu.dk/ase/) and simply multiply the cell.

In [46]:
interactive_view(W_screw_dislo * [1, 1, 10], d_name="1/2<111> screw")

NGLWidget()

### 1/2<111>{110} edge dislocation

As we said before, for edge dislocations burgers vecort $\vec b$ is perpendicular to the dislocation $\vec l$. So here, we have the same glide plane of (110) which fixes the cell Y direction to <110>. X direction now will be along burgers vector <111> and Z dislocation line direction <112>.

In [47]:
from matscipy.dislocation import BCCEdge111Dislocation

W_edge = BCCEdge111Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_edge.axes)

print("Burgers vector:")
print(W_edge.burgers)


Cell orientation:
[[ 1  1  1]
 [ 1 -1  0]
 [ 1  1 -2]]
Burgers vector:
[1.57169453 1.57169453 1.57169453]


In [48]:
W_edge_bulk, W_edge_dislo = W_edge.build_cylinder(radius=15)

print("\nCell vectors:")
print(W_edge_dislo.cell.array)

print("\nBurgers vector lenth:")
print(np.linalg.norm(W_edge.burgers))

interactive_view(W_edge_dislo, scale=0.25, d_name="1/2<111> edge")

disloc SCF 0 |d1-d2|_inf = 0.17503717189385382
disloc SCF 1 |d1-d2|_inf = 0.030669651063355258
disloc SCF 2 |d1-d2|_inf = 0.004376489640269432
disloc SCF 3 |d1-d2|_inf = 0.0008807112000284167
disloc SCF 4 |d1-d2|_inf = 0.00015452837692353505
disloc SCF 5 |d1-d2|_inf = 2.6112647029508107e-05
disloc SCF 6 |d1-d2|_inf = 5.7487804006051135e-06
disloc SCF 7 |d1-d2|_inf = 7.771108773702018e-07

Cell vectors:
[[54.44509545  0.          0.        ]
 [ 0.         53.34508114  0.        ]
 [ 0.          0.          7.69969924]]

Burgers vector lenth:
2.722254772666657


NGLWidget()

It can be seen that the case of edge dislocation the requires more iterations to achieve converged displacement field.

### 1/2<111>{110} mixed dislocation

For mixed dislocation the cell vector are the same as for the screw dislocation. However the displacement vector is applied along <111> direction that is not parallel to the Z direction: [$1\bar 11$] and [$1\bar 1 \bar1$] with an angle of 70.5 degrees between the vectors. This leads to both edge and screw component in the displacement and thus the name _mixed_ dislocation.

In [49]:
from matscipy.dislocation import BCCMixed111Dislocation

W_mixed = BCCMixed111Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_mixed.axes)

print("Burgers vector:")
print(W_mixed.burgers)


Cell orientation:
[[ 1 -1 -2]
 [ 1  1  0]
 [ 1 -1  1]]
Burgers vector:
[ 1.57169453 -1.57169453 -1.57169453]


In [50]:
W_mixed_bulk, W_mixed_dislo = W_mixed.build_cylinder(radius=20)

print("\nCell vectors:")
print(W_mixed_dislo.cell.array)

print("\nBurgers vector lenth:")
print(np.linalg.norm(W_mixed.burgers))

interactive_view(W_mixed_dislo, scale=0.5, d_name="1/2<111> mixed")

disloc SCF 0 |d1-d2|_inf = 0.10539187767041772
disloc SCF 1 |d1-d2|_inf = 0.018105716735450528
disloc SCF 2 |d1-d2|_inf = 0.002853744720166418
disloc SCF 3 |d1-d2|_inf = 0.00044617599789692486
disloc SCF 4 |d1-d2|_inf = 6.968641396387643e-05
disloc SCF 5 |d1-d2|_inf = 1.0882499856867955e-05
disloc SCF 6 |d1-d2|_inf = 1.6994203833053945e-06
disloc SCF 7 |d1-d2|_inf = 2.653822296916353e-07

Cell vectors:
[[61.59759392  0.          0.        ]
 [ 0.         62.235928    0.        ]
 [ 0.          0.          2.72225477]]

Burgers vector lenth:
2.722254772666657


NGLWidget()

### <100>{110} edge 'junction' dislocation

So called junction dislocations with burdgers vector along <100> can be formed in the reactions between more abundant dislocations with burgers vector <111> such as: 

$$
    \frac{1}{2}[1\bar{1}1] + \frac{1}{2}[11\bar{1}] = [100]
$$

They share the same glide plane ad 1/2<111> dislocations and can play important role in impurity segregation. 

In [51]:
from matscipy.dislocation import BCCEdge100110Dislocation

W_100110_edge = BCCEdge100110Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_100110_edge.axes)

print("Burgers vector:")
print(W_100110_edge.burgers)


Cell orientation:
[[ 1  0  0]
 [ 0  1  1]
 [ 0 -1  1]]
Burgers vector:
[3.14338905 0.         0.        ]


In [52]:
W_100110_edge_bulk, W_100110_edge_dislo = W_100110_edge.build_cylinder(radius=17)
interactive_view(W_100110_edge_dislo, scale=0.3, 
                 d_name="<100>{110} edge", d_color=[1, 0, 1])

disloc SCF 0 |d1-d2|_inf = 0.256011923011219
disloc SCF 1 |d1-d2|_inf = 0.06016875789278883
disloc SCF 2 |d1-d2|_inf = 0.008688239875985548
disloc SCF 3 |d1-d2|_inf = 0.002036859690120879
disloc SCF 4 |d1-d2|_inf = 0.00025500641555943615
disloc SCF 5 |d1-d2|_inf = 8.302194247189476e-05
disloc SCF 6 |d1-d2|_inf = 1.6579146270331857e-05
disloc SCF 7 |d1-d2|_inf = 1.7272275882884713e-06
disloc SCF 8 |d1-d2|_inf = 6.753932630942927e-07


NGLWidget()

### <100>{001} edge dislocation

This is the same junction dislocation but lying in a different glide plane.

In [53]:
from matscipy.dislocation import BCCEdge100Dislocation

W_100_edge = BCCEdge100Dislocation(alat, C11, C12, C44, symbol="W")

print("Cell orientation:")
print(W_100_edge.axes)

print("Burgers vector:")
print(W_100_edge.burgers)


Cell orientation:
[[ 1  0  0]
 [ 0  0 -1]
 [ 0  1  0]]
Burgers vector:
[3.14338905 0.         0.        ]


In [54]:
W_100_edge_bulk, W_100_edge_dislo = W_100_edge.build_cylinder(radius=20)
interactive_view(W_100_edge_dislo, scale=0.4, d_name="<100>{001} edge", d_color=[1, 0, 1])

disloc SCF 0 |d1-d2|_inf = 0.29723896079013645
disloc SCF 1 |d1-d2|_inf = 0.03893895819760007
disloc SCF 2 |d1-d2|_inf = 0.011189961805871729
disloc SCF 3 |d1-d2|_inf = 0.001492901691747324
disloc SCF 4 |d1-d2|_inf = 0.00046542738716470744
disloc SCF 5 |d1-d2|_inf = 8.162136711258039e-05
disloc SCF 6 |d1-d2|_inf = 1.3593036747305831e-05
disloc SCF 7 |d1-d2|_inf = 4.370960947186475e-06
disloc SCF 8 |d1-d2|_inf = 5.007713447316431e-07


NGLWidget()

## Face Centered Cubic

As an example of FCC material we will consider Ni and an interatomic potential from [Bonny _et. al._](https://www.tandfonline.com/doi/abs/10.1080/14786430903299824)


In [55]:
eam_calc = EAM("../../tests/FeCuNi.eam.alloy")

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=eam_calc, symbol="Ni")

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 16:10:48      -17.801046*       0.0001
Fitting C_11
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-5.12800014e+00, -2.50333320e+00, -1.41366167e-04,  2.37961959e+00,
        4.63379565e+00]) GPa
Cij (gradient) / GPa    :     244.0654437182867
Error in Cij / GPa      :     4.217968506243491
Correlation coefficient :     0.9995522932872427
Setting C11 (1) to 1.523337 +/- 0.026326


Fitting C_21
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-3.37539886e+00, -1.58459192e+00, -1.41366167e-04,  1.38619385e+00,
        2.58171226e+00]) GPa
Cij (gradient) / GPa    :     148.85008009740656
Error in Cij / GPa      :     6.776719382842348
Correlation coefficient :     0.9969053432709588
Setting C21 (7) to 0.929049 +/- 0.042297


Fitting C_31
Strain array([-0.02, -0.01,  0.  ,  0.01,  0.02])
Stress array([-3.37539886e+00, -1.58459192e+00, -1.41366167e-04,

In [56]:
print(f"{alat:.3f} (Angstrom), {C11:.2f}, {C12:.2f}, {C44:.2f} (GPa)")

3.520 (Angstrom), 244.07, 148.85, 125.62 (GPa)


### 1/2<110>{111} screw dislocation (perfect and dissociated)

In [57]:
from matscipy.dislocation import FCCScrew110Dislocation

Ni_screw = FCCScrew110Dislocation(alat, C11, C12, C44, symbol="Ni")

print("Cell orientation:")
print(Ni_screw.axes)

print("Burgers vector:")
print(Ni_screw.burgers)


Cell orientation:
[[ 1  1 -2]
 [ 1  1  1]
 [ 1 -1  0]]
Burgers vector:
[ 1.76 -1.76  0.  ]


In [58]:
Ni_screw_bulk, Ni_screw_dislo = Ni_screw.build_cylinder(radius=20)


print("\nCell vectors:")
print(Ni_screw_dislo.cell.array)

print("\nBurgers vector lenth:")
print(np.linalg.norm(Ni_screw.burgers))

interactive_view(Ni_screw_dislo, 
                 d_name="1/2<110> screw dislocation line", 
                 d_color=[0, 0, 1])

disloc SCF 0 |d1-d2|_inf = 0.021791458261984697
disloc SCF 1 |d1-d2|_inf = 0.0002936041882874109
disloc SCF 2 |d1-d2|_inf = 4.8928735380371347e-05
disloc SCF 3 |d1-d2|_inf = 1.2696678054050815e-06
disloc SCF 4 |d1-d2|_inf = 1.0764587210498888e-07
disloc SCF 0 |d1-d2|_inf = 0.020321133865591423
disloc SCF 1 |d1-d2|_inf = 0.0006134137472214689
disloc SCF 2 |d1-d2|_inf = 5.9406489367308524e-05
disloc SCF 3 |d1-d2|_inf = 2.948341477265748e-06
disloc SCF 4 |d1-d2|_inf = 8.340714410803862e-08

Cell vectors:
[[60.35542726  0.          0.        ]
 [ 0.         60.96818843  0.        ]
 [ 0.          0.          2.48901587]]

Burgers vector lenth:
2.4890158697766473


NGLWidget()

Due to stable Intrinsic Stacking Fault (ISF) the 1/2<110> dislocations are not stable and dissociate in two partial 1/6<112> Shockley partials separated by stacking fault. For example:

$$
    \frac{1}{2}[1\bar10] \rightarrow \frac{1}{6}[2\bar1\bar1] + \mathrm{ISF} + \frac{1}{2}[1\bar21]
$$

where ISF is intrinsic stacking fault. It is possible to pass a parameter `partial_distance` to create dissociated dislocation. `partial distance` defines separation distance (length of the stacking fault) of two partial dislocations. The value corresponds to number of glide distances the distance in Angstrom be obtaines as `patial_distance * dislocation.glide_distance`.

In [59]:
Ni_screw_bulk, Ni_screw_dislo = Ni_screw.build_cylinder(radius=20, partial_distance=5)
print(f"Expected partial distance: {5 * Ni_screw.glide_distance:.1f} Angstrom")
interactive_view(Ni_screw_dislo, 
                 partial_distance=5 * Ni_screw.glide_distance,
                 d_name="1/6<112> Shockley partial screw")

disloc SCF 0 |d1-d2|_inf = 0.02179145826198453
disloc SCF 1 |d1-d2|_inf = 0.0002936041882874109
disloc SCF 2 |d1-d2|_inf = 4.892873538059339e-05
disloc SCF 3 |d1-d2|_inf = 1.2696678051752307e-06
disloc SCF 4 |d1-d2|_inf = 1.0764587193845543e-07
disloc SCF 0 |d1-d2|_inf = 0.020321133865591506
disloc SCF 1 |d1-d2|_inf = 0.0006134137472214689
disloc SCF 2 |d1-d2|_inf = 5.9406489367308524e-05
disloc SCF 3 |d1-d2|_inf = 2.948341477515548e-06
disloc SCF 4 |d1-d2|_inf = 8.340714444804442e-08
Expected partial distance: 10.778 A


NGLWidget()

### 1/2<110>{111} edge dislocation (perfect and dissociated)

In [60]:
from matscipy.dislocation import FCCEdge110Dislocation

Ni_edge = FCCEdge110Dislocation(alat, C11, C12, C44, symbol="Ni")

print("Cell orientation:")
print(Ni_edge.axes)

print("Burgers vector:")
print(Ni_edge.burgers)


Cell orientation:
[[ 1 -1  0]
 [ 1  1  1]
 [-1 -1  2]]
Burgers vector:
[ 1.76 -1.76  0.  ]


In [61]:
Ni_edge_bulk, Ni_edge_dislo = Ni_edge.build_cylinder(radius=20)

print("\nCell vectors:")
print(Ni_edge_dislo.cell.array)

print("\nBurgers vector lenth:")
print(np.linalg.norm(Ni_edge.burgers))

interactive_view(Ni_edge_dislo, d_name="1/2<110> edge", d_color=[0, 0, 1])

disloc SCF 0 |d1-d2|_inf = 0.04310715442453525
disloc SCF 1 |d1-d2|_inf = 0.0031359890399249857
disloc SCF 2 |d1-d2|_inf = 0.00018078113282300745
disloc SCF 3 |d1-d2|_inf = 1.4501349156037513e-05
disloc SCF 4 |d1-d2|_inf = 9.978781161934513e-07
disloc SCF 0 |d1-d2|_inf = 0.03462421627571943
disloc SCF 1 |d1-d2|_inf = 0.0022123322463467043
disloc SCF 2 |d1-d2|_inf = 0.00021564683900149317
disloc SCF 3 |d1-d2|_inf = 1.1838612354972411e-05
disloc SCF 4 |d1-d2|_inf = 6.324895580611667e-07

Cell vectors:
[[64.71441261  0.          0.        ]
 [ 0.         60.96818843  0.        ]
 [ 0.          0.          4.31110195]]

Burgers vector lenth:
2.4890158697766473


NGLWidget()

In [62]:
Ni_edge_bulk, Ni_edge_dislo = Ni_edge.build_cylinder(radius=20, partial_distance=10)
print(f"Expected partial distance: {10 * Ni_edge.glide_distance:.1f} Angstrom")
interactive_view(Ni_edge_dislo, 
                 partial_distance=10 * Ni_edge.glide_distance, 
                 d_name="1/2<110> Shockley partial edge")

disloc SCF 0 |d1-d2|_inf = 0.04310715442453525
disloc SCF 1 |d1-d2|_inf = 0.0031359890399249857
disloc SCF 2 |d1-d2|_inf = 0.00018078113282300745
disloc SCF 3 |d1-d2|_inf = 1.4501349155759957e-05
disloc SCF 4 |d1-d2|_inf = 9.978781162003902e-07
disloc SCF 0 |d1-d2|_inf = 0.03462421627571943
disloc SCF 1 |d1-d2|_inf = 0.002212332246347065
disloc SCF 2 |d1-d2|_inf = 0.00021564683900149317
disloc SCF 3 |d1-d2|_inf = 1.1838612354972411e-05
disloc SCF 4 |d1-d2|_inf = 6.324895580611667e-07
Expected partial distance: 12.445 A


NGLWidget()

## Diamond 

As an example of dimaond structure we will use Si and potential from [work of D. Holland and M. Marder](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.746)

In [1]:
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import StillingerWeber,\
                                                                Holland_Marder_PRL_80_746_Si
from matscipy.calculators.manybody import Manybody
calc = Manybody(**StillingerWeber(Holland_Marder_PRL_80_746_Si))

# the function accepts any ASE type of calculator
alat, C11, C12, C44 = get_elastic_constants(calculator=calc, symbol="Si")

NameError: name 'get_elastic_constants' is not defined

In [72]:
print(f"{alat:.3f} (Angstrom), {C11:.2f}, {C12:.2f}, {C44:.2f} (GPa)")

5.431 (Angstrom), 201.46, 51.64, 118.18 (GPa)


In [73]:
from matscipy.dislocation import DiamondGlideScrew

Si_screw = DiamondGlideScrew(alat, C11, C12, C44, symbol="Si")

print("Cell orientation:")
print(Si_screw.axes)

print("Burgers vector:")
print(Si_screw.burgers)


Cell orientation:
[[ 1  1 -2]
 [ 1  1  1]
 [ 1 -1  0]]
Burgers vector:
[ 2.71547466 -2.71547466  0.        ]


### 1/2<110>{111} screw dislocation (perfect and dissociated)

In [74]:
Si_screw_bulk, Si_screw_dislo = Si_screw.build_cylinder(radius=20)

print("\nCell vectors:")
print(Si_screw_dislo.cell.array)

print("\nBurgers vector lenth:")
print(np.linalg.norm(Si_screw.burgers))
Si_screw_dislo.wrap()
interactive_view(Si_screw_dislo, 
                 diamond_structure=True, 
                 scale=0.3, 
                 add_bonds=True, 
                 d_name="1/2<110> screw", 
                 d_color=[0, 0, 1])


Cell vectors:
[[66.51527324  0.          0.        ]
 [ 0.         75.25344122  0.        ]
 [ 0.          0.          3.84026109]]

Burgers vector lenth:
3.840261091032201


NGLWidget()

As in FCC structure similar dissociation mechanism exists in diamond structure.

In [75]:
Si_screw_bulk, Si_screw_dislo = Si_screw.build_cylinder(radius=20, partial_distance=5)
Si_screw_dislo.wrap()
print(f"Expected partial distance: {5 * Si_screw.glide_distance:.1f} Angstrom")
interactive_view(Si_screw_dislo, 
                 diamond_structure=True, 
                 scale=0.3, add_bonds=True, 
                 partial_distance=5 * Si_screw.glide_distance, 
                 d_name="1/6<112> 30 degree partial screw") 

Expected partial distance: 16.629 A


NGLWidget()

### 1/2<110>{111} 60 degree screw dislocation (perfect and dissociated)

Due to the particular symmetry of diamond structure, there exist dislocations with 60 degree angle between burgers vector and dislocation line. This dislocation can dissociate in two partials: 30 and 90 degree.

In [79]:
from matscipy.dislocation import DiamondGlide60Degree

Si_60_degree_screw = DiamondGlide60Degree(alat, C11, C12, C44, symbol="Si")

print("Cell orientation:")
print(Si_screw.axes)

print("Burgers vector:")
print(Si_screw.burgers)


Cell orientation:
[[ 1  1 -2]
 [ 1  1  1]
 [ 1 -1  0]]
Burgers vector:
[ 2.71547466 -2.71547466  0.        ]


In [80]:
Si_60_degree_screw_bulk, \
Si_60_degree_screw_dislo = Si_60_degree_screw.build_cylinder(radius=20)

print("\nCell vectors:")
print(Si_60_degree_screw_dislo.cell.array)

print("\nBurgers vector lenth:")
print(np.linalg.norm(Si_60_degree_screw.burgers))
Si_60_degree_screw_dislo.wrap()
interactive_view(Si_60_degree_screw_dislo, 
                 diamond_structure=True, 
                 scale=0.3, add_bonds=True,
                 d_name="1/2<110> 60 degree screw", 
                 d_color=[0, 0, 1])


Cell vectors:
[[66.51527324  0.          0.        ]
 [ 0.         75.25344122  0.        ]
 [ 0.          0.          3.84026109]]

Burgers vector lenth:
3.840261091032201


NGLWidget()

In [87]:
Si_60_degree_screw_bulk, \
Si_60_degree_screw_dislo = Si_60_degree_screw.build_cylinder(radius=20, 
                                                             partial_distance=5)

Si_60_degree_screw_dislo.wrap()
print(f"Expected partial distance: {5 * Si_60_degree_screw.glide_distance:.1f} Angstrom")
interactive_view(Si_60_degree_screw_dislo, 
                 diamond_structure=True, 
                 scale=0.3, 
                 add_bonds=True, 
                 d_color=[1, 0, 1],
                 partial_distance=5 * Si_screw.glide_distance, 
                 d_name=["1/6<112> 90 degree partial screw", 
                         "1/6<112> 30 degree partial screw"])

Expected partial distance: 16.629 A


NGLWidget()